## Create a liquidity pool on Tinyman and trade algorithmically
#### 09.1 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-26


## Setup

In [ ]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [ ]:
from pyteal import *

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

### Create liquidity pool via web page

Tinyman is a dApp for liquidity pools. We are now creating step-by step a liquidity pool via their web interface.

Requirements:
+ Account on the testnet with Algos and one ASA
+ Algorand App set to testnet

Steps:
Tinyman is a dApp for liquidity pools. We are now creating step-by step a liquidity pool via their web interface.

Prepare
+ Go to https://testnet.tinyman.org/
+ Click on "Connect Wallet" top right
+ Choose "Algorand Mobile Wallet"
+ Scan QR code with Algorand Wallet app
+ Choose the acount that contains the ASA

Create pool
+ Click on "pool"
+ "create pair" /  see all /and deselect "hide unverified assets".
+ "Create pool" will create *empty pool* (takes a few rounds = 15sec)

> "Once you create the pool, other users will be able to add liquidity to it, but the pool will be unverified. Creating the pool does not give you additional rights over the pool."

+ Do not forget to confirm the transaction with the mobile phone app

>"To add liquidity, you need to first opt-in your account to the liquidity token of this pool."
+ Click and confirm opt-in transaction on app

+ Now the pool is empty, add liquidity. Select from top. **This defines the initial exchange rate**

+ "Add liquidity" / "Confirm supply"
+  Confirm transactions on app
+  Now check asset holdings --> new asset "Tinyman Pool"

Use the pool
+ You and others can go to "SWAP" and access the pool (remember: deselect "hide unverified assets")
  + Try to start with very small amounts
  + Increase amount, see how price impact changes exchange rate
  












### Python and Tinyman 

In [ ]:
# install the github packages
!pip install git+https://github.com/tinymanorg/tinyman-py-sdk.git

In [ ]:
# import packages
from tinyman.v1.pools import Pool
from tinyman.assets import Asset
from tinyman.utils import wait_for_confirmation
from tinyman.v1.client import TinymanClient
from tinyman.v1.client import TinymanTestnetClient

In [ ]:
import algosdk
import base64
from algosdk import account, mnemonic
import json
from algosdk import template
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.transaction import PaymentTxn, LogicSig
from pyteal import *

In [ ]:
# Connect to the Tinyman Client
client = TinymanClient(
    algod_client=algod_client,
    validator_app_id=62368684,    # this is the testnet app ID (do not change)
)

In [ ]:
# Get suggest params
sp = algod_client.suggested_params()

## Connect to Tinyman

In [ ]:
account = MyAlgo

In [ ]:
# Check if the account is opted into Tinyman dApp
if(not client.is_opted_in(account['public'])):
    print('Account not opted into app, opting in now..')
    transaction_group = client.prepare_app_optin_transactions(account['public'])
    for i, txn in enumerate(transaction_group.transactions):
        if txn.sender == account['public']:
            transaction_group.signed_transactions[i] = txn.sign(account['private'])
    txid = client.algod.send_transactions(transaction_group.signed_transactions)
    wait_for_confirmation(txid)


In [ ]:
# define function for logic sig of any pool
import importlib.resources
import tinyman.v1
from tinyman.utils import get_program
_contracts = json.loads(importlib.resources.read_text(tinyman.v1, 'asc.json'))

pool_logicsig_def = _contracts['contracts']['pool_logicsig']['logic']

validator_app_def = _contracts['contracts']['validator_app']

def get_pool_logicsig(validator_app_id, asset1_id, asset2_id):
    assets = [asset1_id, asset2_id]
    asset_id_1 = max(assets)
    asset_id_2 = min(assets)
    program_bytes = get_program(pool_logicsig_def, variables=dict(
        validator_app_id=validator_app_id,
        asset_id_1=asset_id_1,
        asset_id_2=asset_id_2,
    ))
    return LogicSig(program=program_bytes)

In [ ]:
# Have a look at Algoexplorer to see the App that we have opted into
# Click on "Apps"
print("https://testnet.algoexplorer.io/address/"+account['public'])

#### Get your asset holdings

In [ ]:
asset_holdings_df(algod_client,account['public'])

In [ ]:
WSC_asset_id = 70627982        # <------------ CHANGE THIS!! use your ID

client = TinymanTestnetClient(user_address=account['public'])
# By default all subsequent operations are on behalf of user_address

# Fetch our two assets of interest
WinterSchoolCoin = client.fetch_asset(WSC_asset_id)
ALGO = client.fetch_asset(0)

# Fetch the pool we will work with
pool = client.fetch_pool(WinterSchoolCoin, ALGO)

In [ ]:
print(pool.address)    # noting to see YET
print(pool.asset1)
print(pool.asset2)

#### Manually add liquidity

**If we execute the next cell without putting liquidity inside the pool, we will get an error that states division by 0. This is because we have to provide liquidity to our pool.**

8. We click on add liquidity on Tiniyman and we select the exchange rate that we want for our coin. We confirm and sign the transaction.

9. Now we are able to check the pool swap prices. 

In [ ]:
# Get a quote for a swap of 1 ALGO to WUSI with 1% slippage tolerance
# Note: Algo in microAlgo
quote = pool.fetch_fixed_input_swap_quote(ALGO(1_000_000), slippage=0.01)
print(quote)
print(f'WUSI per ALGO: {quote.price}')
print(f'WUSI per ALGO (worst case): {quote.price_with_slippage}')
# you can check if the swap rate is the same on the exchange

## Assumptions made for the pool
- The exchange will be permissionless, meaning that anyone can use it.
- The exchange is fully decentralized and immutable - nobody can update/delete/freeze the Pools or assets.
- The exchange is non-custodial. Only the Poolers have access to their shares of the Pools.
- The AMM will be a constant product market maker, like Uniswap.
- The Pools hold assets in a 50/50 ratio.
- The AMM will allow exchange between pairs of Algorand Standard Assets (ASAs) or ASA and Algo.
- The ASAs should have high liquidity - a large total supply (not NFTs or collectables).
- The minimum swap/mint/burn size is expected to be 1000 microunits.
-A percentage based fee is charged on every trade, called the swap fee. This fee increases a Pool’s liquidity and benefits liquidity providers (Poolers).

#### Useful resources 

https://github.com/tinymanorg/tinyman-py-sdk --> Tinyman's github

https://docs.tinyman.org/design-doc#bootstrap-process --> documentation to set the liquidity pool. Very useful if you want further details.

https://app.tinyman.org/#/pool --> Tinyman app



#### Or bootstrap pool manually

In [ ]:
# This is a function to bootstrap you pool directly from python. 

from os import name
import algosdk
from algosdk.future.transaction import ApplicationOptInTxn, PaymentTxn, AssetCreateTxn, AssetOptInTxn
from algosdk.v2client.algod import AlgodClient

from tinyman.utils import int_to_bytes, TransactionGroup



def prepare_bootstrap_transactions(validator_app_id, asset1_id, asset2_id, asset1_unit_name, asset2_unit_name, sender, suggested_params):
    pool_logicsig = get_pool_logicsig(validator_app_id, asset1_id, asset2_id)
    pool_address = pool_logicsig.address()

    assert(asset1_id > asset2_id)

    if asset2_id == 0:
        asset2_unit_name = 'ALGO'

    txns = [
        PaymentTxn(
            sender=sender,
            sp=suggested_params,
            receiver=pool_address,
            amt=961000 if asset2_id > 0 else 860000,     # TX fee
            note='fee',
        ),
        ApplicationOptInTxn(
            sender=pool_address,
            sp=suggested_params,
            index=validator_app_id,
            app_args=['bootstrap', int_to_bytes(asset1_id), int_to_bytes(asset2_id)],
            foreign_assets=[asset1_id] if asset2_id == 0 else [asset1_id, asset2_id],
        ),
        AssetCreateTxn(
            sender=pool_address,
            sp=suggested_params,
            total=0xFFFFFFFFFFFFFFFF,
            decimals=6,
            unit_name='TMPOOL11',
            asset_name=f'TinymanPool1.1 {asset1_unit_name}-{asset2_unit_name}',
            url='https://tinyman.org',
            default_frozen=False,
        ),
        AssetOptInTxn(
            sender=pool_address,
            sp=suggested_params,
            index=asset1_id,
        ),
    ]
    if asset2_id > 0:
        txns += [
            AssetOptInTxn(
                sender=pool_address,
                sp=suggested_params,
                index=asset2_id,
            )
        ]
    txn_group = TransactionGroup(txns)
    txn_group.sign_with_logicisg(pool_logicsig)
    return txn_group

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
txgroup = prepare_bootstrap_transactions(62368684, 
                                         WSC_asset_id, 0, 
                                         "TEMP", "ALGO", MyAlgo['public'], sp)

In [ ]:
txgroup.transactions

In [ ]:
# Step 2: sign
txgroup.sign_with_private_key(MyAlgo['public'], MyAlgo['private'])

# Step 3: send
txid = algod_client.send_transactions(txgroup.signed_transactions)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

### opt into pool

In [ ]:
## not needed
def prepare_app_optin_transactions(validator_app_id, sender, suggested_params):
    txn = ApplicationOptInTxn(
        sender=sender,
        sp=suggested_params,
        index=validator_app_id,
    )
    txn_group = TransactionGroup([txn])
    return txn_group


def prepare_asset_optin_transactions(asset_id, sender, suggested_params):
    txn = AssetOptInTxn(
        sender=sender,
        sp=suggested_params,
        index=asset_id,
    )
    txn_group = TransactionGroup([txn])
    return txn_group

In [ ]:
txgroup = pool.prepare_liquidity_asset_optin_transactions()
txgroup.sign_with_private_key(MyAlgo['public'], MyAlgo['private'])
txgroup_submitted = algod_client.send_transactions(txgroup.signed_transactions)
print(txgroup_submitted)

### Fund pool

In [ ]:
TEMP = client.fetch_asset(WSC_asset_id)
ALGO = client.fetch_asset(0)
pool = client.fetch_pool(TEMP, ALGO)
pool_id = pool.liquidity_asset.id
amounts_in={}
amt_1 = 20
amt_2 = 1000

In [ ]:
from algosdk.future.transaction import ApplicationNoOpTxn, PaymentTxn, AssetTransferTxn

def prepare_mint_transactions(validator_app_id, asset1_id, asset2_id, liquidity_asset_id, asset1_amount, asset2_amount, liquidity_asset_amount, sender, suggested_params):
    pool_logicsig = get_pool_logicsig(validator_app_id, asset1_id, asset2_id)
    pool_address = pool_logicsig.address()

    txns = [
        PaymentTxn(
            sender=sender,
            sp=suggested_params,
            receiver=pool_address,
            amt=2000,
            note='fee',
        ),
        ApplicationNoOpTxn(
            sender=pool_address,
            sp=suggested_params,
            index=validator_app_id,
            app_args=['mint'],
            accounts=[sender],
            foreign_assets=[asset1_id, liquidity_asset_id] if asset2_id == 0 else [asset1_id, asset2_id, liquidity_asset_id],
        ),
        AssetTransferTxn(
            sender=sender,
            sp=suggested_params,
            receiver=pool_address,
            amt=int(asset1_amount),
            index=asset1_id,
        ),
        AssetTransferTxn(
            sender=sender,
            sp=suggested_params,
            receiver=pool_address,
            amt=int(asset2_amount),
            index=asset2_id,
        ) if asset2_id != 0 else PaymentTxn(
            sender=sender,
            sp=suggested_params,
            receiver=pool_address,
            amt=int(asset2_amount),
        ),
        AssetTransferTxn(
            sender=pool_address,
            sp=suggested_params,
            receiver=sender,
            amt=int(liquidity_asset_amount),
            index=liquidity_asset_id,
        ),
    ]
    txn_group = TransactionGroup(txns)
    txn_group.sign_with_logicisg(pool_logicsig)
    return txn_group

In [ ]:
#prepare_mint_transactions(validator_app_id, asset1_id, asset2_id, liquidity_asset_id, asset1_amount, asset2_amount, liquidity_asset_amount, sender, suggested_params)
params = algod_client.suggested_params()
txgroup = prepare_mint_transactions(62368684, WSC_asset_id, 0, pool_id, amt_1, amt_2, 1, MyAlgo['public'],params)

In [ ]:
txgroup.sign_with_private_key(MyAlgo['public'], MyAlgo['private'])

In [ ]:
txgroup.signed_transactions

In [ ]:
txgroup_submitted = algod_client.send_transactions(txgroup.signed_transactions)

In [ ]:
quote = pool.fetch_mint_quote(TEMP(1),  slippage=0.01)

### Appendix: integration with MyAlgo

1. Create a new account on MyAlgo https://wallet.myalgo.com/home.
  Be sure to save the mnemonic, and switch from mainet to testnet, and fund the account https://bank.testnet.algorand.network/.

2. On the top left there is MyAlgo logo, click on it and go on asset manager

3. Now click on create asset and create your coin, you can avoid to put the metadata hash and the URL.

4. Now you have created a coin, let's create a pool. We use Tinyman to create this, that works similar to Uniswap. [You need to use chrome] https://testnet.tinyman.org/#/swap?asset_in=0. We need to define some new terminology here:
- Pools - liquidity pools holding 2 assets
- Poolers - users who provide assets to the Pools
- Swappers - users who trade/swap assets through Pools
- Assets - Algorand Standard Assets (ASAs) and Algo
- Liquidity Token - An ASA that represents a share of the value of a Pool

### Appendix: Tinyman code

The code is here on the blockchain: https://testnet.algoexplorer.io/application/21580889

Here on Github: https://github.com/tinymanorg/tinyman-contracts-v1

